In [1]:
import pandas as pd
import numpy as np

import requests
import pickle
import joblib

import time
from selenium import webdriver

from flask import Flask, render_template, request, jsonify, Response
from scipy.optimize import linear_sum_assignment

from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

app = Flask(__name__)
@app.route("/", methods=['GET', 'POST'])
def home():
    
    #Need to install requests package for python
    #easy_install requests
    
    # Set the request parameters
    url = 'https://dev97589.service-now.com/api/now/table/u_endava_deployment?sysparm_limit=1000'
    
    # Eg. User name="admin", Password="admin" for this code sample.
    user = 'admin'
    pwd = 'HwaV9p1GrMkL'
    
    # Set proper headers
    headers = {"Content-Type":"application/json","Accept":"application/json"}
    
    # Do the HTTP request
    response = requests.get(url, auth=(user, pwd), headers=headers )
    
    # Check for HTTP codes other than 200
    if response.status_code != 200: 
        print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
        exit()
    
    # Decode the JSON response into a dictionary and use the data
    data = response.json()
    
    inc_df = pd.DataFrame(data["result"])
    inc_df = inc_df[['sys_id', 'u_date', 'u_description', 'u_team', 'u_priority', 'u_assigned_to', 'u_status']]
    inc_df = inc_df[inc_df['u_status'] == 'Open']
    
    def ticket_pred(df):
    
        vocab_size = 5000
        embedding_dim = 32 #64
        max_length = 60
        trunc_type = 'post'
        padding_type = 'post'
        oov_tok = '<OOV>'
        training_portion = 0.8
        
        # Load the saved team model.
        team_model = load_model('lstm_model.h5')
        
        # loading
        with open('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
        
        inc_df = df
        txt = inc_df['u_description']
        # tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
        tokenizer.fit_on_texts(txt)
        seq = tokenizer.texts_to_sequences(txt)
        padded = pad_sequences(seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
        pred = team_model.predict(padded)
        labels = ['Code', 'Connectivity', 'Permission', 'Administrative', 'Network']
        
        team_pred = labels[np.argmax(pred)-1]
        inc_df['u_team'] = team_pred
        
        def job_alloc(team):
        
            resource_name = pd.read_csv("resource_name_master.csv", index_col=0)
            job_allo_master = pd.read_csv("job_allocation_master.csv", index_col=0)
            job_allo_master.index.name = None
            
            team_alloc = team
            
            row_ind, col_ind   = linear_sum_assignment(job_allo_master, maximize=False)
            available_resourse_df = pd.DataFrame({'team' : list(np.array(job_allo_master.index)), \
                                           'resource' : list(np.array(job_allo_master.columns)[col_ind])})
            available_resourse = available_resourse_df.loc[available_resourse_df['team'] == team_alloc, 'resource'].iloc[0]
            job_allo_master.loc[ [team_alloc] ,  available_resourse] = job_allo_master.loc[ [team_alloc] ,  available_resourse] + 1
            job_allo_master.to_csv('job_allocation_master.csv', index=True) 
            
            avail_resource = resource_name.loc[ [team_alloc] ,  available_resourse].iloc[0]
            
            return avail_resource
        
        assigned_to = job_alloc(team_pred)
        inc_df['u_assigned_to'] = assigned_to
        
        # Load the saved priority model
        priority_model = joblib.load('priority_model.jl')
        
        priority_pred = priority_model.predict(txt)
        
        if priority_pred[0] == '0':
            priority = "Low Priority"
        else:
            priority = "High Priority"
            
        inc_df['u_priority'] = priority
        inc_df['u_status'] = 'Pending'
        
        #Need to install requests package for python
        #easy_install requests
        
        # Set the request parameters
        sys_id = inc_df.iloc[0]['sys_id']
        print(sys_id)
        url = 'https://dev97589.service-now.com/api/now/table/u_endava_deployment/' + sys_id
        url = str(url)
        print(url)
        
        # Eg. User name="admin", Password="admin" for this code sample.
        user = 'admin'
        pwd = 'HwaV9p1GrMkL'
        
        # Set proper headers
        headers = {"Content-Type":"application/json","Accept":"application/json"}
        
        load = {'u_team': team_pred, 'u_priority': priority, 'u_assigned_to': assigned_to, 'u_status': 'Pending'}
        
        # Do the HTTP request
        response = requests.put(url, auth=(user, pwd), headers=headers, json=load )
        
        # Check for HTTP codes other than 200
        if response.status_code != 200: 
            print('Status:', response.status_code, 'Headers:', response.headers, 'Error Response:',response.json())
            exit()
        
        # Decode the JSON response into a dictionary and use the data
        data = response.json()
    
        # return Response(inc_df.to_json(orient="records"), mimetype='application/json')
        return inc_df
    
    
    if inc_df.empty == False:
        
        ticket_predicted = ticket_pred(inc_df)
        print("Ticket allocation is successful!!!")
        
    else:
        print('No new tickets to process!!!')
    
        # return Response(inc_df.to_json(orient="records"), mimetype='application/json')
        return inc_df.to_html()
        # return data
    

if __name__ == "__main__":
    app.run()
    


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Dec/2021 10:29:25] "GET / HTTP/1.1" 200 -


No new tickets to process!!!


127.0.0.1 - - [04/Dec/2021 10:29:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 10:29:27] "GET /favicon.ico HTTP/1.1" 404 -


No new tickets to process!!!


...

No new tickets to process!!!


127.0.0.1 - - [04/Dec/2021 11:24:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 11:24:42] "GET /favicon.ico HTTP/1.1" 404 -


No new tickets to process!!!


127.0.0.1 - - [04/Dec/2021 11:29:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 11:29:35] "GET /favicon.ico HTTP/1.1" 404 -


No new tickets to process!!!
